# MIMIC Pre-Processing Example
The output of the [MIMIC Pre-Processing Example](./MimicPreprocessingExample.ipynb) notebook generates a series of irregularly spaced timestamped dynamic data features.  This notebook demonstrates how to group those dynamic features into one or more regularly spaced time windows.

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import os
from sklearn.preprocessing import LabelEncoder 

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
dataDirStr = '/Users/gmessier/data/mimic-1.4/'   # MIMIC CSV file location.
cacheDirStr = '/Users/gmessier/data/mimic-1.4/cache/'  # Cache directory for intermediate files.

## Load Data
---

In [3]:
events = pd.read_parquet(cacheDirStr + 'ElapsedTimeEvents.parquet')

In [4]:
events

Time  ServiceType  AdmitType  GscVerbal  \
SubjectId                                                               
2         73554    0 days 00:00:00          NaN        2.0        NaN   
          368      0 days 00:19:42          3.0        NaN        NaN   
          6461661  0 days 01:16:00          NaN        NaN        NaN   
          6461662  0 days 01:26:00          NaN        NaN        NaN   
          6461663  0 days 01:56:00          NaN        NaN        NaN   
...                            ...          ...        ...        ...   
99999     22559585 2 days 10:41:52          NaN        NaN        8.0   
          22559586 2 days 10:41:52          NaN        NaN        NaN   
          6440611  2 days 11:10:52          NaN        NaN        NaN   
          6440612  2 days 11:10:52          NaN        NaN        NaN   
          25146197 2 days 11:10:52          NaN        NaN        NaN   

                    GscMotor  GscEyes  SystolicBloodPressure  HeartRate  \
SubjectId                                                                 
2         73554          NaN      NaN                    NaN        NaN   
          368            NaN      NaN                    NaN        NaN   
          6461661        NaN      NaN                    NaN      148.0   
          6461662        NaN      NaN                    NaN      131.0   
          6461663        NaN      NaN                    NaN      144.0   
...                      ...      ...                    ...        ...   
99999     22559585       NaN      NaN                    NaN        NaN   
          22559586       8.0      NaN                    NaN        NaN   
          6440611        NaN      NaN                    NaN       86.0   
          6440612        NaN      NaN                  120.0        NaN   
          25146197       NaN      NaN                    NaN        NaN   

                    Temperature  BloodO2  Urine  Age   Aids  Cancer  Death  
SubjectId                                                                   
2         73554             NaN      NaN    NaN    0  False   False  False  
          368               NaN      NaN    NaN    0  False   False  False  
          6461661           NaN      NaN    NaN    0  False   False  False  
          6461662           NaN      NaN    NaN    0  False   False  False  
          6461663           NaN      NaN    NaN    0  False   False  False  
...                         ...      ...    ...  ...    ...     ...    ...  
99999     22559585          NaN      NaN    NaN   63  False   False  False  
          22559586          NaN      NaN    NaN   63  False   False  False  
          6440611           NaN      NaN    NaN   63  False   False  False  
          6440612           NaN      NaN    NaN   63  False   False  False  
          25146197          NaN      NaN   80.0   63  False   False  False  

[25952164 rows x 15 columns]

### Feature Types

In [5]:
# Categorical
catFtrs = [ 'ServiceType', 'AdmitType', 'GscVerbal', 'GscMotor', 'GscEyes' ] 

# Continous
contFtrs = [ 'SystolicBloodPressure', 'HeartRate', 'Temperature', 'BloodO2', 'Urine' ]

# Static
staticFtrs = [  'Age', 'Aids', 'Cancer', 'Death' ]

## Define Observation Window
---

In [6]:
tO = pd.Timedelta(f'{24*60*60}S') # Observation window length, T_O (seconds).
events = events.loc[events.Time <= tO]

In [7]:
events

Time  ServiceType  AdmitType  GscVerbal  \
SubjectId                                                               
2         73554    0 days 00:00:00          NaN        2.0        NaN   
          368      0 days 00:19:42          3.0        NaN        NaN   
          6461661  0 days 01:16:00          NaN        NaN        NaN   
          6461662  0 days 01:26:00          NaN        NaN        NaN   
          6461663  0 days 01:56:00          NaN        NaN        NaN   
...                            ...          ...        ...        ...   
99995     6439849  0 days 23:30:39          NaN        NaN        NaN   
          6439302  0 days 23:30:39          NaN        NaN        NaN   
          25766653 0 days 23:30:39          NaN        NaN        NaN   
99999     66685    0 days 00:00:00         11.0        NaN        NaN   
          129909   0 days 05:25:52          NaN        1.0        NaN   

                    GscMotor  GscEyes  SystolicBloodPressure  HeartRate  \
SubjectId                                                                 
2         73554          NaN      NaN                    NaN        NaN   
          368            NaN      NaN                    NaN        NaN   
          6461661        NaN      NaN                    NaN      148.0   
          6461662        NaN      NaN                    NaN      131.0   
          6461663        NaN      NaN                    NaN      144.0   
...                      ...      ...                    ...        ...   
99995     6439849        NaN      NaN                  115.0        NaN   
          6439302        NaN      NaN                    NaN       72.0   
          25766653       NaN      NaN                    NaN        NaN   
99999     66685          NaN      NaN                    NaN        NaN   
          129909         NaN      NaN                    NaN        NaN   

                    Temperature  BloodO2  Urine  Age   Aids  Cancer  Death  
SubjectId                                                                   
2         73554             NaN      NaN    NaN    0  False   False  False  
          368               NaN      NaN    NaN    0  False   False  False  
          6461661           NaN      NaN    NaN    0  False   False  False  
          6461662           NaN      NaN    NaN    0  False   False  False  
          6461663           NaN      NaN    NaN    0  False   False  False  
...                         ...      ...    ...  ...    ...     ...    ...  
99995     6439849           NaN      NaN    NaN   89  False   False   True  
          6439302           NaN      NaN    NaN   89  False   False   True  
          25766653          NaN      NaN   30.0   89  False   False   True  
99999     66685             NaN      NaN    NaN   63  False   False  False  
          129909            NaN      NaN    NaN   63  False   False  False  

[3123791 rows x 15 columns]

## One Hot Encode Categorical Features
---

In [8]:
oneHot = pd.DataFrame(index=events.index)

In [9]:
for c in tqdm(catFtrs):
    oneHot = pd.concat([ oneHot, pd.get_dummies(events[c],prefix=c) ], axis=1)

  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
oneHotCols = list(oneHot.columns)

In [11]:
events = pd.concat([ events.drop(columns=catFtrs), oneHot ], axis=1)

## Resample Into Time Bins
---

In [12]:
tB = f'{60*60}S' # Bin size (seconds).

### One Hot Encoded Features
Sum up how many events fall into each time bin.

In [13]:
eventsBin = events[['Time']+oneHotCols].groupby(level=0).resample(on='Time',rule=tB).sum()

### Continuous Features
Take the average of the feature value for each time bin.

In [14]:
eventsBinNew = events[['Time']+contFtrs].groupby(level=0).resample(on='Time',rule=tB).mean()

In [15]:
eventsBin = pd.concat([eventsBin, eventsBinNew], axis=1)

### Static Features
Copy static features across all time bins since they don't change.

In [16]:
staticTbl = events[staticFtrs].groupby(level=0).apply(lambda x: x.iloc[0,:])

In [17]:
idx = eventsBin.index.get_level_values(0)

In [18]:
for c in staticFtrs:
    eventsBin[c] = staticTbl.loc[idx,c].values

## Save Windowed Data

In [19]:
eventsBin

ServiceType_0.0  ServiceType_1.0  ServiceType_2.0  \
SubjectId Time                                                                 
2         0 days 00:00:00                0                0                0   
          0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
3         0 days 00:00:00                0                0                0   
          0 days 01:00:00                0                0                0   
...                                    ...              ...              ...   
99999     0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
          0 days 03:00:00                0                0                0   
          0 days 04:00:00                0                0                0   
          0 days 05:00:00                0                0                0   

                           ServiceType_3.0  ServiceType_4.0  ServiceType_5.0  \
SubjectId Time                                                                 
2         0 days 00:00:00                1                0                0   
          0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
3         0 days 00:00:00                0                0                0   
          0 days 01:00:00                0                0                0   
...                                    ...              ...              ...   
99999     0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
          0 days 03:00:00                0                0                0   
          0 days 04:00:00                0                0                0   
          0 days 05:00:00                0                0                0   

                           ServiceType_6.0  ServiceType_7.0  ServiceType_8.0  \
SubjectId Time                                                                 
2         0 days 00:00:00                0                0                0   
          0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
3         0 days 00:00:00                0                0                0   
          0 days 01:00:00                0                0                0   
...                                    ...              ...              ...   
99999     0 days 01:00:00                0                0                0   
          0 days 02:00:00                0                0                0   
          0 days 03:00:00                0                0                0   
          0 days 04:00:00                0                0                0   
          0 days 05:00:00                0                0                0   

                           ServiceType_9.0  ...  GscEyes_7.0  \
SubjectId Time                              ...                
2         0 days 00:00:00                0  ...            0   
          0 days 01:00:00                0  ...            0   
          0 days 02:00:00                0  ...            0   
3         0 days 00:00:00                1  ...            0   
          0 days 01:00:00                0  ...            0   
...                                    ...  ...          ...   
99999     0 days 01:00:00                0  ...            0   
          0 days 02:00:00                0  ...            0   
          0 days 03:00:00                0  ...            0   
          0 days 04:00:00                0  ...            0   
          0 days 05:00:00                0  ...            0   

                           SystolicBloodPressure  HeartRate  Temperatur

In [20]:
eventsBin.to_parquet(cacheDirStr + 'ElapsedTimeEventsBinned.parquet')